In [2]:
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

_____

# Process Data methods

In [3]:
def processDataLSA(pathTXT):

    acceptance_pb = []
    acc_best_sol = []
    temp = []
    best_y = []
    curr_y = []
    runtime = 0
    iteration_nbr = 0


    with open(pathTXT, "r") as file:
        dimension = file.readline().count('BEST x')
        best_xs = np.zeros((dimension, 0))
        curr_xs = np.zeros((dimension, 0))

        for line in file:
            if(line.startswith("Runtime")):
                runtime = float(re.sub("[^0-9^.]", "", line))
            elif(line.startswith("Nbr")):
                iteration_nbr = int(re.sub("[^0-9]", "", line))
            else:
                final = 5
                a = line.split("|")
                a = [i.strip() for i in a][:-1]
                acceptance_pb.append(float(a[0]))
                acc_best_sol.append(a[1])
                temp.append(float(a[2]))
                best_y.append(float(a[3]))
                curr_y.append(float(a[4]))

                best_xs_temp = np.zeros((dimension, 1))
                curr_xs_temp = np.zeros((dimension, 1))
                for i in range(dimension):
                    index = i*2
                    best_xs_temp[i] = float(a[final+index])
                    curr_xs_temp[i] = float(a[final+index+1])
                best_xs = np.concatenate((best_xs, best_xs_temp), axis = 1)
                curr_xs = np.concatenate((curr_xs, curr_xs_temp), axis = 1)
                
    acceptance_pb = np.asarray(acceptance_pb)
    acc_best_sol = np.asarray(acc_best_sol)
    temp = np.asarray(temp)
    best_y = np.asarray(best_y)
    curr_y = np.asarray(curr_y)
    
    return acceptance_pb, acc_best_sol, temp, best_y, curr_y, best_xs, curr_xs, runtime, iteration_nbr

In [4]:
def processMarkovLen(init):
    temp = []
    zeros = 0
    for i, v in enumerate(init):
        if(v == 0):
            zeros = zeros+1
            v = 2
        if(i>1):
            if(v==1):
                temp.append(init[i-1])
    temp.append(v)
    
    final = list(np.ones(int(temp[0]+1+zeros))*temp[0])
    for i in range(len(temp)-1):
        final += list(np.linspace(int(temp[i]), int(temp[i+1]), int(temp[i+1]+1)))[1:]
    return np.asarray(final)

In [5]:
def processDataDSA(pathTXT):
    acceptance_pb = []
    acc_best_sol = []
    temp = []
    density = []
    markov_len = []
    best_y = []
    curr_y = []
    runtime = 0
    iteration_nbr = 0
    
    with open(pathTXT, "r") as file:
        dimension = file.readline().count('BEST x')
        best_xs = np.zeros((dimension, 0))
        curr_xs = np.zeros((dimension, 0))

        for line in file:
            if(line.startswith("Runtime")):
                runtime = float(re.sub("[^0-9^.]", "", line))
            elif(line.startswith("Nbr")):
                iteration_nbr = int(re.sub("[^0-9]", "", line))
            else:
                final = 7
                a = line.split("|")
                a = [i.strip() for i in a][:-1]
                acceptance_pb.append(float(a[0]))
                acc_best_sol.append(a[1])
                temp.append(float(a[2]))
                density.append(float(a[3]))
                markov_len.append(float(a[4]))
                best_y.append(float(a[5]))
                curr_y.append(float(a[6]))

                best_xs_temp = np.zeros((dimension, 1))
                curr_xs_temp = np.zeros((dimension, 1))
                for i in range(dimension):
                    index = i*2
                    best_xs_temp[i] = float(a[final+index])
                    curr_xs_temp[i] = float(a[final+index+1])
                best_xs = np.concatenate((best_xs, best_xs_temp), axis = 1)
                curr_xs = np.concatenate((curr_xs, curr_xs_temp), axis = 1)
                
    acceptance_pb = np.asarray(acceptance_pb)
    acc_best_sol = np.asarray(acc_best_sol)
    temp = np.asarray(temp)
    best_y = np.asarray(best_y)
    curr_y = np.asarray(curr_y)
    
    density = np.asarray(density)
    markov_len = processMarkovLen(markov_len)[:len(temp)]
    
    
    return acceptance_pb, acc_best_sol, temp, density, markov_len, best_y, curr_y, best_xs, curr_xs, runtime, iteration_nbr

In [6]:
#path = "../java/SimulatedAnnealing/GraphData/DSA_MinFunction.txt"
#acceptance_pb, acc_best_sol, temp, density, markov_len, best_y, curr_y, best_xs, curr_xs, runtime, iteration_nbr = processDataDSA(path)
#print(len(markov_len), len(temp))

In [7]:
import pandas as pd

def processData4D_panda(pathTXT):
    a = []
    tab = []
    i = 0
    with open(pathTXT, "r") as file:
        for line in file:
            if(line.startswith("Runtime")):
                runtime = float(re.sub("[^0-9^.]", "", line))
            elif(line.startswith("Nbr")):
                iteration_nbr = int(re.sub("[^0-9]", "", line))
            else:
                a = line.split("|")
                a = [i.strip() for i in a][:-1]
                if(i!=0):
                    a = [float(i) for i in a]
                tab.append(a)
            i+=1
            
    titles = tab[0]    
    col_to_del = titles[:5]
    dtf = pd.DataFrame(data=tab[1:],columns=titles)
    afterdrop_all = dtf.drop(col_to_del, axis = 1)

    col_to_del = titles[:5]+titles[5::2]
    afterdrop_cur = dtf.drop(col_to_del, axis = 1)
    
    col_to_del = titles[:5]+titles[6::2]
    afterdrop_bes = dtf.drop(col_to_del, axis = 1)
    
    isCG = dtf['Control-G?']==1.0
    col_to_del = titles[:5]+titles[5::2]
    afterdrop_curCG = dtf[isCG].drop(col_to_del, axis = 1)
    
    return afterdrop_all, afterdrop_cur, afterdrop_bes, afterdrop_curCG

In [8]:
#data, data_cur, data_best, data_curCG = processData4D_panda('../java/SimulatedAnnealing/GraphData/DSA_MinFunction4D.txt')
#data_curCG.head()

In [9]:
import pandas as pd

def processData_stocha(pathTXT):
    a = []
    tab = []
    i = 0
    with open(pathTXT, "r") as file:
        for line in file:
            a = line.split("|")
            a = [i.strip() for i in a][:-1]
            if(i!=0):
                a = [float(i) for i in a]
            tab.append(a)
            i+=1
            
    titles = tab[0]    
    dtf = pd.DataFrame(data=tab[1:],columns=titles)
    
    col_to_del = titles[:5]+titles[6::2]
    afterdrop_bes = dtf.drop(col_to_del, axis = 1)
    
    return afterdrop_bes

In [9]:
data_mult = processData_stocha('../java/SimulatedAnnealing/GraphData/DSA_MinFunction4D_Stocha.txt')
data_mult

,BEST y,BEST x,BEST x2,BEST x3
0,0.509352,-0.592515,-0.835429,-0.785650
1,0.509210,-0.751112,-0.633323,-0.543274
2,0.509322,-0.853368,-0.818937,-0.899342
3,0.509549,-0.934003,-0.942177,-0.678712
4,0.509218,-0.739728,-0.894205,-0.673323
5,0.509283,-1.010039,-0.766428,-0.630949
6,0.509292,-0.989060,-0.549152,-0.775527
7,0.509440,-1.008318,-0.501024,-0.845229
8,0.509304,-0.939832,-0.711170,-0.535135
9,0.509310,-0.687184,-0.767545,-0.901594
